# HLS Foundation Model Finetuning notebook

This notebook demonstrates the steps to fintune the HLS foundation model (A.K.A Prithvi) which is trained using HLSL30 and HLSS30 datasets. 

Note: Entierty of this notebook is desigend to work well within the AWS sagemaker environment. AWS sagemaker environment access for your account can be found using http://smd-ai-workshop-creds-webapp.s3-website-us-east-1.amazonaws.com/.

![HLS Training](../images/HLS-training.png)

In [ ]:
# Install required packages
!pip install -r requirements.txt

# Create directories needed for data, model, and config preparations
!mkdir datasets
!mkdir models
!mkdir configs

## Dataset preparation

For this hands-on session, Burn Scars example will be used for fine-tuning. All of the data and pre-trained models are available in Huggingface. Huggingface packages and git will be utilized to download, and prepare datasets and pretrained models.


Note: Git Large File Storage (git LFS) is utilized to download larger files from huggingface.

In [ ]:
# Install git lfs
! sudo apt-get install git-lfs; git lfs install

### Download HLS Burn Scars dataset from Huggingface: https://huggingface.co/datasets/ibm-nasa-geospatial/hls_burn_scars

In [ ]:
! cd datasets; git clone https://huggingface.co/datasets/ibm-nasa-geospatial/hls_burn_scars; tar -xvzf hls_burn_scars/hls_burn_scars.tar.gz 

## Download config and Pre-trained model

The HLS Foundation Model (pre-trained model), and configuration for Burn Scars downstream task are available in Huggingface. We use `huggingface_hub` python package to download the files locally.

In [ ]:
# Define constants
BUCKET_NAME = '<your-bucket-name>' # Replace this with the bucket name available from http://smd-ai-workshop-creds-webapp.s3-website-us-east-1.amazonaws.com/ 
CONFIG_PATH = './configs'
DATASET_PATH = './datasets'
MODEL_PATH = './models'

Note: The configuration file in Huggingface have place holders and won't be directly usable for fine-tuning. Placeholder values need to be updated for individual usecases.

In [ ]:
# Download config file from huggingface
from huggingface_hub import hf_hub_download

hf_hub_download(repo_id="ibm-nasa-geospatial/Prithvi-100M-burn-scar", filename="burn_scars_Prithvi_100M.py", local_dir='./configs')

In [ ]:
# Download pre-trained model file from huggingface
hf_hub_download(repo_id="ibm-nasa-geospatial/Prithvi-100M", filename="Prithvi_100M.pt", local_dir='./models')

*Warning: * Before running the remaining cells please update the details in the configuration file as mentioned below:

1. Update line number 13 from `data_root = '<path to data root>'` to `data_root = '/opt/ml/data/'`. This is the base of our data inside of sagemaker.
2. Update line number 41 from `pretrained_weights_path = '<path to pretrained weights>'` to `pretrained_weights_path = f"{data_root}/models/Prithvi_100M.pt"`. This provides the pre-trained model path to the train script.
3. Update line number 53 from `experiment = '<experiment name>'` to `experiment = 'burn_scars'` or your choice of experiment name.
4. Update line number 54 from `project_dir = '<project directory name>'` to `project_dir = 'v1'` or your choice of project directory name. 
5. Save the config file.

In [ ]:
# Prepare sagemaker session with files uploaded to s3 bucket
import sagemaker

sagemaker_session = sagemaker.Session()
train_images = sagemaker_session.upload_data(path='datasets/training', bucket=BUCKET_NAME, key_prefix='data/training')
val_images = sagemaker_session.upload_data(path='datasets/validation', bucket=BUCKET_NAME, key_prefix='data/validation')
test_images = sagemaker_session.upload_data(path='datasets/validation', bucket=BUCKET_NAME, key_prefix='data/test')

In [ ]:
# Rename configuration file name to user specific filename
import os

identifier = '<your choice of identifier>' # Please update this with an identifier

config_filename = 'configs/burn_scars_Prithvi_100M.py'
new_config_filename = f"configs/{identifier}-burn_scars_Prithvi_100M.py"
os.rename(config_filename, new_config_filename)

In [ ]:
# Upload config files to s3 bucket
configs = sagemaker_session.upload_data(path=new_config_filename, bucket=BUCKET_NAME, key_prefix='data/configs')
models = sagemaker_session.upload_data(path='models/Prithvi_100M.pt', bucket=BUCKET_NAME, key_prefix='data/models')


Note: For HLS Foundation Model, MMCV and MMSEG were used. These libraries use pytorch underneath them for training, data distribution etc. However, these packages are not available in sagemaker by default. Thus, custom script training is required. Sagemaker utilizes Docker for custom training scripts. If interested, the code included in the image we are using for training (637423382292.dkr.ecr.us-west-2.amazonaws.com/sagemaker_hls:latest) is bundled with this repository, and the train script used is `train.py`.

The current HLS Foundation model fits in a single NVIDIA Tesla V100 GPU (16GB VRAM). Hence, `ml.p3.2xlarge` instance is used for training.

In [ ]:
# Setup variables for training using sagemaker
from datetime import time
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator


name = f'{identifier}-sagemaker'
role = get_execution_role()
input_s3_uri = f"s3://{BUCKET_NAME}/data"

environment_variables = {
    'CONFIG_FILE': f"/opt/ml/data/{new_config_filename}",
    'MODEL_DIR': "/opt/ml/models/",
    'MODEL_NAME': f"{identifier}-workshop.pth",
    'S3_URL': input_s3_uri,
    'ROLE_ARN': role,
    'ROLE_NAME': role.split('/')[-1],
    'EVENT_TYPE': 'burn_scars',
    'VERSION': 'v1'
}

ecr_container_url = '637423382292.dkr.ecr.us-west-2.amazonaws.com/sagemaker_hls:latest'
sagemaker_role = 'SageMaker-ExecutionRole-20240206T151814'

instance_type = 'ml.p3.2xlarge'

instance_count = 1
memory_volume = 50

In [ ]:
# Establish an estimator (model) using sagemaker and the configurations from the previous cell.
estimator = Estimator(image_uri=ecr_container_url,
                      role=get_execution_role(),
                      base_job_name=name,
                      instance_count=1,
                      environment=environment_variables,
                      instance_type=instance_type)


In [ ]:
# Start training
estimator.fit()